# Notebook Dedicated to Save the TTree Data information in Numpy Format

In [1]:
import uproot
import numpy as np
import random 

import glob
import re
import os

import seaborn as sn
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm

# Useful Functions

In [2]:
def magnitude(vector): 
    #Function to Compute the radial position of events
    x = vector[:,0]
    y = vector[:,1]
    z = vector[:,2]

    r = np.sqrt(x**2 + y**2 + z**2)
    r = r.astype(np.float32)
    return r

In [3]:
def random_unit_vector(seed=None):

    '''
    Function that generates a normalized vector with a random direction

    Parameters:
    - seed: Value to reproduce the generated result

    return: A normalized 3-vector
    '''

    
    # Create a random generator with optional seed
    rng = np.random.default_rng(seed)
    # Generate 3 components from a normal distribution
    vec = rng.normal(size=3)
    # Normalize to unit length
    vec /= np.linalg.norm(vec)
    
    return vec

# Read Files: Get the file list with the full directories of .root files

In [4]:
main_dir = 'E:/Data/solars/mc/'
solarNue_fdir = 'solar_Nue/High Stat/solar_Nue*.root'
tl_208_fdir = '208tl/High Stat/tl208*.root'

# ---- Define the dataset to contruct ------
construct_train = True
construct_test = False

are_different = (construct_train != construct_test)

if not(are_different):
    print('Chose only one dataset to construct!')

if construct_train and are_different:
    print('working the train dataset')
    
    solar_Nue_flist = glob.glob(main_dir + solarNue_fdir )[:-2] #Selected the elements in list of dir to be used for training
    tl_208_flist = glob.glob(main_dir + tl_208_fdir)[:-2] #Selected the elements in list of dir to be used for training
       
if construct_test and are_different:
    print('working the test dataset')
    
    solar_Nue_flist = glob.glob(main_dir + solarNue_fdir )[-2:] #Selected the elements in list of dir to be used for training
    tl_208_flist = glob.glob(main_dir + tl_208_fdir)[-2:] #Selected the elements in list of dir to be used for training

working the train dataset


In [5]:
solar_Nue_flist

['E:/Data/solars/mc/solar_Nue/High Stat\\solar_Nue_mc_analysis_0.root',
 'E:/Data/solars/mc/solar_Nue/High Stat\\solar_Nue_mc_analysis_1.root',
 'E:/Data/solars/mc/solar_Nue/High Stat\\solar_Nue_mc_analysis_10.root',
 'E:/Data/solars/mc/solar_Nue/High Stat\\solar_Nue_mc_analysis_11.root',
 'E:/Data/solars/mc/solar_Nue/High Stat\\solar_Nue_mc_analysis_12.root',
 'E:/Data/solars/mc/solar_Nue/High Stat\\solar_Nue_mc_analysis_13.root',
 'E:/Data/solars/mc/solar_Nue/High Stat\\solar_Nue_mc_analysis_14.root',
 'E:/Data/solars/mc/solar_Nue/High Stat\\solar_Nue_mc_analysis_15.root',
 'E:/Data/solars/mc/solar_Nue/High Stat\\solar_Nue_mc_analysis_16.root',
 'E:/Data/solars/mc/solar_Nue/High Stat\\solar_Nue_mc_analysis_17.root',
 'E:/Data/solars/mc/solar_Nue/High Stat\\solar_Nue_mc_analysis_18.root',
 'E:/Data/solars/mc/solar_Nue/High Stat\\solar_Nue_mc_analysis_19.root',
 'E:/Data/solars/mc/solar_Nue/High Stat\\solar_Nue_mc_analysis_2.root',
 'E:/Data/solars/mc/solar_Nue/High Stat\\solar_Nue_mc_

In [6]:
tl_208_flist

['E:/Data/solars/mc/208tl/High Stat\\tl208_mc_analysis_0.root',
 'E:/Data/solars/mc/208tl/High Stat\\tl208_mc_analysis_1.root',
 'E:/Data/solars/mc/208tl/High Stat\\tl208_mc_analysis_10.root',
 'E:/Data/solars/mc/208tl/High Stat\\tl208_mc_analysis_11.root',
 'E:/Data/solars/mc/208tl/High Stat\\tl208_mc_analysis_12.root',
 'E:/Data/solars/mc/208tl/High Stat\\tl208_mc_analysis_13.root',
 'E:/Data/solars/mc/208tl/High Stat\\tl208_mc_analysis_14.root',
 'E:/Data/solars/mc/208tl/High Stat\\tl208_mc_analysis_15.root',
 'E:/Data/solars/mc/208tl/High Stat\\tl208_mc_analysis_16.root',
 'E:/Data/solars/mc/208tl/High Stat\\tl208_mc_analysis_17.root',
 'E:/Data/solars/mc/208tl/High Stat\\tl208_mc_analysis_18.root',
 'E:/Data/solars/mc/208tl/High Stat\\tl208_mc_analysis_19.root',
 'E:/Data/solars/mc/208tl/High Stat\\tl208_mc_analysis_2.root',
 'E:/Data/solars/mc/208tl/High Stat\\tl208_mc_analysis_20.root',
 'E:/Data/solars/mc/208tl/High Stat\\tl208_mc_analysis_21.root',
 'E:/Data/solars/mc/208tl/Hi

# Analysis of .root files: Extract Observables of Interest

In [7]:
# ============ Data Cuts Settings ============
energy_inf_cut = 2.5
energy_sup_cut = 12

posr_cut = 5500

time_res_inf_cut = -1
time_res_sup_cut = 5

nhits_cut = 20 #Nhits cut to be applied after the time residual cuts

# ============ Output Directies ============

if construct_train and are_different:
    out_file_dir = f'E:/Data/solars/mc/ML Data/np_arrays/Train/E_{energy_inf_cut}_{energy_sup_cut}_MeV_R_{posr_cut}_mm_Delta_{time_res_inf_cut}_{time_res_sup_cut}_ns/'
    os.makedirs(out_file_dir, exist_ok=True)


if construct_test and are_different:  
    out_file_dir = f'E:/Data/solars/mc/ML Data/np_arrays/Test/E_{energy_inf_cut}_{energy_sup_cut}_MeV_R_{posr_cut}_mm_Delta_{time_res_inf_cut}_{time_res_sup_cut}_ns/'
    os.makedirs(out_file_dir, exist_ok=True)

## Solars Analysis

In [30]:
f = uproot.open(solar_Nue_flist[0])
f.keys()

pmt_data = f['pmt;1']
data = f['T;31']

pmt_xyz = np.array(pmt_data['pmt_pos_xyz'])
hitpmt_id = np.array(data['hit_pmtid'])

In [25]:
a.append(b[0])
a

[array([1, 2, 3])]

In [16]:
np.array(a)

array([[1, 2, 3],
       [1, 2, 3]])

In [46]:
data_type_fname = 'solarNue'

# -------------------------------------------------------

for fi_dx, input_file_dir in enumerate(solar_Nue_flist):
    print(f'In file {input_file_dir}')
    
    #Load the Data:
    load_data = uproot.open(input_file_dir)
    
    #select the tree of event data and PMT info
    TTree_data_name = load_data.keys()[0]
    TTree_pmt_info_name = load_data.keys()[-1]
    
    event_data = load_data[TTree_data_name]
    pmt_data = load_data[TTree_pmt_info_name]
    
    #event info to be used:
    var_event_list = ['evtid', 'energy', 'position', 'mc_momentum',
                      'hit_pmtid', 'hit_residual', 'hit_pmtQHS']  #list the name of the varibles to be extracted and used for the solarnu analysis.
    
    #pmt info to be used
    var_pmt_list = ['pmt_id', 'pmt_pos_xyz', 'pmt_type']
    
    #Observables to save
    var_name_save_list = ['evtid', 'energy', 'posr', 'hit_residual', 'hit_pmtQHS', 'position']
    multi_cos_alpha = [] #create the empty list of the cos_alpha for the multiple PMTs record
    multi_hit_pmt_xyz = [] #create the empty list to save the hit pmt coordinates
    n_init_evs = []  # Initial Nº of Solar Nue MC events
    n_final_evs = [] # Initial Nº of Solar Nue MC events
    
    # Extract the variables with the name of the var_event_list in numpy.array from the .root file
    observables = {}
    
    # Load variables form the Event Info TTree Branch
    for var_name_i in var_event_list:
        observables[var_name_i] = np.array(event_data[var_name_i])
        
    # ============= Count the initial Nº of events =============
    evtid = observables['evtid']
    evIDi_unique = []  #Empty list to be filled with the unique (non-redundant) values of the initial evIDs. len(evIDi_unique) = Nº of events

    N_data = len(evtid)
    indices_to_delete = [] #Indiced to remove from the observable arrays due to nhits cut

    data_break_i = [0]  #array whose elements are the index from where an event start and end.
    #Loop to extract the indices where evtid breaks
    for i_dx in range(N_data - 1):
        if evtid[i_dx] != evtid[i_dx+1]:
            data_break_i.append(i_dx + 1)
            
    data_break_i.append(N_data - 1)

    N_terms = len(data_break_i)
    
    for i_dx in range(N_terms - 2):
        init_i = data_break_i[i_dx]
        final_i = data_break_i[i_dx + 1]
        evIDi_unique.append(evtid[init_i: final_i][0])

    N_evs = len(evIDi_unique)
    print(f'Nº of initial events: {N_evs}')
    n_init_evs.append(N_evs)
    # ===========================================================
    
    # Compute posr
    observables['posr'] = magnitude(observables['position'])
    
    # Load the PMT info TTree Branch
    try:
        #Try to extract the info from the PMT Branch
        for var_name_i in var_pmt_list:
            observables[var_name_i] = np.array(pmt_data[var_name_i])
            
    except uproot.KeyInFileError as e:
        #if the Branch doesnt exist, then use the PMT info from other file that we know contais the PMT info Branch
        load_data = uproot.open('E:/Data/solars/mc/solar_Nue/High Stat/solar_Nue_mc_analysis_0.root')
        pmt_data = load_data['pmt;1']
        for var_name_i in var_pmt_list:
            observables[var_name_i] = np.array(pmt_data[var_name_i])
    
    # Filtering of valid PMT id through PMT type
    pmt_type_condition = (observables['pmt_type'] == 1)
    pmt_id_valid = observables['pmt_id'][pmt_type_condition]
    
    # general cut conditions
    hit_pmt_id_condition = np.in1d(observables['hit_pmtid'], pmt_id_valid)
    energy_condition = (observables['energy'] >= energy_inf_cut) & (observables['energy'] <= energy_sup_cut)
    time_res_condition = (observables['hit_residual'] >= time_res_inf_cut) & (observables['hit_residual'] <= time_res_sup_cut)
    posr_condition = (observables['posr'] <= posr_cut)
    
    general_condition = hit_pmt_id_condition & energy_condition & time_res_condition & posr_condition
    
    # Apply the general cut conditions to observables
    for var_name_i in var_event_list:
        observables[var_name_i] = observables[var_name_i][np.array(general_condition)]
    observables['posr'] = observables['posr'][general_condition]

    #print(f'selected energies form {input_file_dir}: {observables['energy']} with shape {observables['energy'].shape}')

# ======== Counts of final events ======
    print('performing nhits cuts after general selections')
    evtid = observables['evtid']
    evIDf_unique = []  #Empty list to be filled with the unique (non-redundant) values of the final evIDs. len(evIDf_unique) = Nº of final events
    N_data = len(evtid)
    indices_to_delete = [] #Indiced to remove from the observable arrays due to nhits cut

    data_break_i = [0]  #array whose elements are the index from where an event start and end.
    #Loop to extract the indices where evtid breaks
    for i_dx in range(N_data - 1):
        if evtid[i_dx] != evtid[i_dx+1]:
            data_break_i.append(i_dx + 1)
            
    data_break_i.append(N_data - 1)

    N_terms = len(data_break_i)

    for i_dx in range(N_terms - 2):
        init_i = data_break_i[i_dx]
        final_i = data_break_i[i_dx + 1]

        evIDf_unique.append(evtid[init_i: final_i][0]) #Count the Number of final events
            
    N_evs = len(evIDf_unique)
    print(f'Nº of final events: {N_evs}')
    n_final_evs.append(N_evs)

    # ====================================================================
    
    # Add to dictionary of observables the keys init evs and final evs
    observables['n_init_evs'] = n_init_evs
    observables['n_final_evs'] = n_final_evs
    
    # add the number of events to the save observable list
    full_observable_save_list = var_name_save_list + ['n_init_evs', 'n_final_evs']
    
    print(f'saving observables {full_observable_save_list}')
    
    for var_name_i in  full_observable_save_list:
        np.save(out_file_dir + data_type_fname + '_' + var_name_i + f'_{fi_dx}.npy', observables[var_name_i])

    #cos_alpha computation
    #The cos_alpha computation must be done by parts using a split in the data.
    
    N_samples = len(observables['hit_residual'])

    print('Saving PMT xyz Coordinates and Computing cos_alpha')
    
    for sample_idx in range(N_samples):
        
        sun_dir = observables['mc_momentum'][sample_idx]
        pmt_hit_id = observables['hit_pmtid'][sample_idx]
        pmt_hit_xyz = observables['pmt_pos_xyz'][pmt_hit_id]

        norm1 = np.linalg.norm(sun_dir)
        norm2 = np.linalg.norm(pmt_hit_xyz)
        
        sun_dir = sun_dir / norm1
        sun_dir = sun_dir.astype(np.float32)
        
        pmt_hit_xyz = pmt_hit_xyz / norm2
        pmt_hit_xyz = pmt_hit_xyz.astype(np.float32)
    
        dot_prod = np.dot(sun_dir, pmt_hit_xyz)
        cos_alpha = dot_prod

        multi_hit_pmt_xyz.append(pmt_hit_xyz)
        multi_cos_alpha.append(cos_alpha)

    multi_cos_alpha = np.array(multi_cos_alpha)
    multi_hit_pmt_xyz = np.array(multi_hit_pmt_xyz)

    print('saving hit PMT xyz coordinates')
    np.save(out_file_dir + data_type_fname + '_hitpmt_xyz' + f'_{fi_dx}.npy', multi_hit_pmt_xyz)
    
    print('saving cos_alpha')
    np.save(out_file_dir + data_type_fname + '_cos_alpha' + f'_{fi_dx}.npy', multi_cos_alpha)

print('Analysis Done!')

In file E:/Data/solars/mc/solar_Nue/High Stat\solar_Nue_mc_analysis_0.root
Nº of initial events: 77653
performing nhits cuts after general selections
Nº of final events: 27219
saving observables ['evtid', 'energy', 'posr', 'hit_residual', 'hit_pmtQHS', 'position', 'n_init_evs', 'n_final_evs']
Saving PMT xyz Coordinates and Computing cos_alpha
saving hit PMT xyz coordinates
saving cos_alpha
In file E:/Data/solars/mc/solar_Nue/High Stat\solar_Nue_mc_analysis_1.root
Nº of initial events: 77830
performing nhits cuts after general selections
Nº of final events: 27227
saving observables ['evtid', 'energy', 'posr', 'hit_residual', 'hit_pmtQHS', 'position', 'n_init_evs', 'n_final_evs']
Saving PMT xyz Coordinates and Computing cos_alpha
saving hit PMT xyz coordinates
saving cos_alpha
In file E:/Data/solars/mc/solar_Nue/High Stat\solar_Nue_mc_analysis_10.root
Nº of initial events: 76822
performing nhits cuts after general selections
Nº of final events: 27050
saving observables ['evtid', 'energy'

## 208Tl Analysis

In [47]:
data_type_fname = '208Tl'

# -------------------------------------------------------

for fi_dx, input_file_dir in enumerate(tl_208_flist):
    print(f'In file {input_file_dir}')
    
    #Load the Data:
    load_data = uproot.open(input_file_dir)  #ROOT File
    #sun_dir = np.load(out_file_dir + f'sun_dir_{fi_dx}.npy') #Solar direction from MC Solar Nue Data to replace 208Tl event
    #solar_evID = np.load(out_file_dir + f'solarNue_evtid_{fi_dx}.npy') #Solar evID to cut on 208Tl data due to direction replacement
    
    #select the tree of event data and PMT info
    TTree_data_name = load_data.keys()[0]
    TTree_pmt_info_name = load_data.keys()[-1]
    
    event_data = load_data[TTree_data_name]
    pmt_data = load_data[TTree_pmt_info_name]
    
    #event info to be used:
    var_event_list = ['evtid', 'energy', 'position', 'mc_momentum',
                      'hit_pmtid', 'hit_residual', 'hit_pmtQHS' ]  #list the name of the varibles to be extracted and used for the solarnu analysis.
    
    #pmt info to be used
    var_pmt_list = ['pmt_id', 'pmt_pos_xyz', 'pmt_type']
    
    #Observables to save
    var_name_save_list = ['evtid', 'energy', 'posr', 'hit_residual', 'hit_pmtQHS', 'position']
    multi_cos_alpha = [] #create the empty list of the cos_alpha for the multiple PMTs record
    multi_hit_pmt_xyz = [] #create the empty list to save the hit pmt coordinates
    n_init_evs = []  # Initial Nº of 208Tl  MC events
    n_final_evs = [] # Initial Nº of 208Tl MC events
    
    # Extract the variables with the name of the var_event_list in numpy.array from the .root file
    observables = {}
    
    # Load variables form the Event Info TTree Branch
    for var_name_i in var_event_list:
        observables[var_name_i] = np.array(event_data[var_name_i])

    # ============= Count the initial Nº of events =============
    evtid = observables['evtid']
    evIDi_unique = []  #Empty list to be filled with the unique (non-redundant) values of the initial evIDs. len(evIDi_unique) = Nº of events

    N_data = len(evtid)
    indices_to_delete = [] #Indiced to remove from the observable arrays due to nhits cut

    data_break_i = [0]  #array whose elements are the index from where an event start and end.
    #Loop to extract the indices where evtid breaks
    for i_dx in range(N_data - 1):
        if evtid[i_dx] != evtid[i_dx+1]:
            data_break_i.append(i_dx + 1)
            
    data_break_i.append(N_data - 1)

    N_terms = len(data_break_i)
    
    for i_dx in range(N_terms - 2):
        init_i = data_break_i[i_dx]
        final_i = data_break_i[i_dx + 1]
        evIDi_unique.append(evtid[init_i: final_i][0])

    N_evs = len(evIDi_unique)
    print(f'Nº of initial events: {N_evs}')
    n_init_evs.append(N_evs)
    # ===========================================================
    
    # posr Calculation
    observables['posr'] = magnitude(observables['position'])
    
    # Cargar variables del árbol de PMTs
    try:
        #Try to extract the info from the PMT Branch
        for var_name_i in var_pmt_list:
            observables[var_name_i] = np.array(pmt_data[var_name_i])
            
    except uproot.KeyInFileError as e:
        #if the Branch doesnt exist, then use the PMT info from other file that we know contais the PMT info Branch
        load_data = uproot.open('E:/Data/solars/mc/solar_Nue/High Stat/solar_Nue_mc_analysis_0.root')
        pmt_data = load_data['pmt;1']
        for var_name_i in var_pmt_list:
            observables[var_name_i] = np.array(pmt_data[var_name_i])
    
    # Filtering of valid PMT id through PMT type
    pmt_type_condition = (observables['pmt_type'] == 1)
    pmt_id_valid = observables['pmt_id'][pmt_type_condition]
    
    # general cut conditions
    hit_pmt_id_condition = np.in1d(observables['hit_pmtid'], pmt_id_valid)
    energy_condition = (observables['energy'] >= energy_inf_cut) & (observables['energy'] <= energy_sup_cut)
    time_res_condition = (observables['hit_residual'] >= time_res_inf_cut) & (observables['hit_residual'] <= time_res_sup_cut)
    posr_condition = (observables['posr'] <= posr_cut)
    
    general_condition = hit_pmt_id_condition & energy_condition & time_res_condition & posr_condition
    
    # Apply the general cut conditions to observables
    for var_name_i in var_event_list:
        observables[var_name_i] = observables[var_name_i][general_condition]
    observables['posr'] = observables['posr'][general_condition]

    print('Redefining 208Tl direction')
    #208Tl direction Redefinition with random rotations of vectors: ----------------------------------------------------------------
    
    # Verify where the data recorded during and event breaks
    tl_break_i = [0]
    tl208_evID = observables['evtid']

    tl_N_data = len(tl208_evID)
    for tl_i in range(tl_N_data - 1):
        if tl208_evID[tl_i] != tl208_evID[tl_i+1]:
            tl_break_i.append(tl_i+1) #append the index where data breaks

    #Redefine every repeated value direction with the new rotated direction
    print(f'True 208Tl momentum {observables['mc_momentum']}')
    N_breaks = len(tl_break_i)
    for i_dx in range(N_breaks - 1):
        random_vec = random_unit_vector(seed = 42)
        i_tl = tl_break_i[i_dx]
        f_tl = tl_break_i[i_dx + 1]
        observables['mc_momentum'][i_tl:f_tl] = random_vec
        #observables['mc_momentum'][i_tl:f_tl] = apply_random_rotation_per_vector([observables['mc_momentum'][i_tl:f_tl][0]], seed = 42, verbose = False)
    observables['mc_momentum'][tl_break_i[-1]:] = random_vec
    #observables['mc_momentum'][tl_break_i[-1]:] = apply_random_rotation_per_vector([observables['mc_momentum'][-1]], seed = 42, verbose = False)
    
    #208Tl direction Redefinition with the direction of solar Nue Mc: ----------------------------------------------------------------
    
    #Match the Nº of events on Tl208 with the solar Nue to correctly replace the 208Tl direction with the sun direction
    #1) Verify where the data recorded during and event breaks;
    #2) Evaluate for each event record the number of entries;
    #3) Evaluate if N_Nue < or > than N_208Tl, and cut depending on this;

    #print('evaluating where the indices Break')
    #1)
    #solar_break_i = [0] #List with the indices where the ev data breaks
    #tl_break_i = [0]
    #tl208_evID = observables['evtid']

    #solar_N_data = len(solar_evID)
    #for solar_i in range(solar_N_data - 1):
    #    if solar_evID[solar_i] != solar_evID[solar_i+1]:
    #        solar_break_i.append(solar_i+1) #append the index where data breaks
        
    #tl_N_data = len(tl208_evID)
    #for tl_i in range(tl_N_data - 1):
    #    if tl208_evID[tl_i] != tl208_evID[tl_i+1]:
    #        tl_break_i.append(tl_i+1) #append the index where data breaks

    #2) and 3)
    #N_solar_evs = len(solar_break_i)
    #N_tl_evs = len(tl_break_i)

    #print(f'N_solar_evs = {N_solar_evs}')
    #print(f'N_208Tl_evs = {N_tl_evs}')

    #if N_solar_evs > N_tl_evs:
    #    solar_break_i = solar_break_i[0:tl_N_data]        
    
    #if  N_tl_evs > N_solar_evs:
    #    tl_break_i = tl_break_i[0:N_solar_evs]

    #print(f'True 208Tl momentum {observables['mc_momentum']}')

    #N_breaks = len(tl_break_i)
    #for i_dx in range(N_breaks - 1):
    #    i_solar = solar_break_i[i_dx]
    #    f_solar = solar_break_i[i_dx + 1]
    #    i_tl = tl_break_i[i_dx]
    #    f_tl = tl_break_i[i_dx + 1]
    #    observables['mc_momentum'][i_tl:f_tl] = sun_dir[i_solar:f_solar][0]
    #observables['mc_momentum'][tl_break_i[-1]:-1] = sun_dir[-1]
    
    #print(f'new 208Tl momentum {observables['mc_momentum']}')
    #print('208Tl direction redefined!')

    #print(f'selected energies form {input_file_dir}: {observables['energy']} with shape {observables['energy'].shape}')

    # ======== Counts of final events ======
    print('performing nhits cuts after general selection')
    evtid = observables['evtid']
    evIDf_unique = []  #Empty list to be filled with the unique (non-redundant) values of the final evIDs. len(evIDf_unique) = Nº of final events
    N_data = len(evtid)
    indices_to_delete = [] #Indiced to remove from the observable arrays due to nhits cut

    data_break_i = [0]  #array whose elements are the index from where an event start and end.
    #Loop to extract the indices where evtid breaks
    for i_dx in range(N_data - 1):
        if evtid[i_dx] != evtid[i_dx+1]:
            data_break_i.append(i_dx + 1)
            
    data_break_i.append(N_data - 1)

    N_terms = len(data_break_i)

    for i_dx in range(N_terms - 2):
        init_i = data_break_i[i_dx]
        final_i = data_break_i[i_dx + 1]
        
        evIDf_unique.append(evtid[init_i: final_i][0]) #Count the Number of final events
           
    N_evs = len(evIDf_unique)
    print(f'Nº of final events: {N_evs}')
    n_final_evs.append(N_evs)

    # ====================================================================
    
    #Add to dictionary of observables the keys init evs and final evs
    observables['n_init_evs'] = n_init_evs
    observables['n_final_evs'] = n_final_evs
    
    # Save desired variables
    full_observable_save_list = var_name_save_list + ['n_init_evs', 'n_final_evs']
    
    print(f'saving observables {full_observable_save_list}')
    
    for var_name_i in  full_observable_save_list:
        np.save(out_file_dir + data_type_fname + '_' + var_name_i + f'_{fi_dx}.npy', observables[var_name_i])

    #cos_alpha computation
    #The cos_alpha computation must be done by parts using a split in the data.
    
    N_samples = len(observables['hit_residual'])

    print('Computing cos_alpha')
    
    for sample_idx in range(N_samples):
        
        fake_dir = observables['mc_momentum'][sample_idx]
        pmt_hit_id = observables['hit_pmtid'][sample_idx]
        pmt_hit_xyz = observables['pmt_pos_xyz'][pmt_hit_id]

        norm1 = np.linalg.norm(fake_dir)
        norm2 = np.linalg.norm(pmt_hit_xyz)
        
        fake_dir = fake_dir / norm1
        sun_dir = sun_dir
        
        pmt_hit_xyz = pmt_hit_xyz / norm2
        pmt_hit_xyz = pmt_hit_xyz
    
        dot_prod = np.dot(fake_dir, pmt_hit_xyz)
        cos_alpha = dot_prod
    
        multi_cos_alpha.append(cos_alpha)
        multi_hit_pmt_xyz.append(pmt_hit_xyz)
        
    multi_hit_pmt_xyz = np.array(multi_hit_pmt_xyz)
    multi_cos_alpha = np.array(multi_cos_alpha)

    print('saving hit PMT xyz coordinates')
    np.save(out_file_dir + data_type_fname + '_hitpmt_xyz' + f'_{fi_dx}.npy', multi_hit_pmt_xyz)
    
    print('saving cos_alpha')
    np.save(out_file_dir + data_type_fname + '_cos_alpha' + f'_{fi_dx}.npy', multi_cos_alpha)

print('Analysis Done!')

In file E:/Data/solars/mc/208tl/High Stat\tl208_mc_analysis_0.root
Nº of initial events: 47658
Redefining 208Tl direction
True 208Tl momentum [[-1.14824581 -0.08212933  0.34899449]
 [-1.14824581 -0.08212933  0.34899449]
 [-1.14824581 -0.08212933  0.34899449]
 ...
 [ 1.15627158  0.15217964  1.41467392]
 [ 1.15627158  0.15217964  1.41467392]
 [ 1.15627158  0.15217964  1.41467392]]
performing nhits cuts after general selection
Nº of final events: 25765
saving observables ['evtid', 'energy', 'posr', 'hit_residual', 'hit_pmtQHS', 'position', 'n_init_evs', 'n_final_evs']
Computing cos_alpha
saving hit PMT xyz coordinates
saving cos_alpha
In file E:/Data/solars/mc/208tl/High Stat\tl208_mc_analysis_1.root
Nº of initial events: 47848
Redefining 208Tl direction
True 208Tl momentum [[ 0.44767275 -0.06471057  0.01904605]
 [ 0.44767275 -0.06471057  0.01904605]
 [ 0.44767275 -0.06471057  0.01904605]
 ...
 [ 0.40401375 -0.10754116  1.71217012]
 [ 0.40401375 -0.10754116  1.71217012]
 [ 0.40401375 -0.1